# Bioinformatics Project 2025 - Motif CNN & GO Prediction

**Course:** GRS34806 Deep Learning

**Authors:** ............

**Date:**



## Importing Libraries

Clean the code last time! also the redundant libraries.

In [1]:
import os
import sys
from pathlib import Path

! git clone https://git.wur.nl/bioinformatics/grs34806-deep-learning-project-data.git -q
! git clone https://github.com/maussn/GRS34806-project.git -q
os.chdir(Path('grs34806-deep-learning-project-data'))

In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import precision_recall_fscore_support
from datetime import datetime
import sys
import os
from pathlib import Path
import random
import seaborn as sns

## Data I/O & Tokenisation

In [3]:
# 1. read() --------------------------------------------------------------------
def read(seqfile: str, posfile: str) -> tuple[list, list]:
    """Read file with sequences and file with positive cases into lists.

    :param seqfile: file with sequences
    :type seqfile: str
    :param posfile: file with positive cases (annotated with function)
    :type posfile: str
    :return: two lists, first with sequences and second with boolean labels
    :rtype: tuple[list, list]
    """
    idlist = []
    datalist = []
    labellist = []
    with open(seqfile, 'r') as f:
        for line in f.readlines():
            line = line.rstrip().split('\t')
            idlist.append(line[0])
            datalist.append(line[1])
            labellist.append(False)
    with open(posfile, 'r') as f:
        for line in f.readlines():
            id = line.rstrip()
            try:
                i = idlist.index(id)
                labellist[i] = True
            except ValueError:
                continue
    return datalist, labellist


# 2. split_labelled() ----------------------------------------------------------
def split_labelled(datalist: list, labellist: list):
    """Return two separate sequence lists: positives & negatives."""
    pos_datalist = []
    neg_datalist = []
    for i, label in enumerate(labellist):
        if label:
            pos_datalist.append(datalist[i])
        else:
            neg_datalist.append(datalist[i])
    return pos_datalist, neg_datalist


# 3. remove_sequences() -----
def remove_sequences(datalist: list, fraction=0.5):
    """Randomly keeps half of the list"""
    random.shuffle(datalist)
    keep = round(len(datalist) * fraction)
    return datalist[:keep]


# 4. fuse_sequence_lists() ------------
def fuse_sequence_lists(pos_datalist: list, neg_datalist: list):
    """Merge postives and negetaves into one list + label"""
    pos_labels = [True for _ in pos_datalist]
    neg_labels = [False for _ in neg_datalist]
    datalist = pos_datalist + neg_datalist
    labellist = pos_labels + neg_labels
    return datalist, labellist


# 5. generate_train_test() --------
def generate_train_test(datalist: list, labellist: list, fraction: float=0.8):
    """Split up dataset in training set and test set

    :param datalist: list with sequences
    :type datalist: list
    :param labellist: list with labels
    :type labellist: list
    :param ratio: fraction to be added to the training set, remainder is added to the test set, defaults to 0.8
    :type ratio: float, optional
    :return: four lists, first two the training data and labels, second two the test data and labels
    :rtype: tuple[list, list, list, list]
    """
    c = list(zip(datalist, labellist))
    random.shuffle(c)
    datalist[:], labellist[:] = zip(*c)
    i = round(len(datalist) * fraction)
    traindatalist = datalist[:i]
    trainlabellist = labellist[:i]
    testdatalist = datalist[i:]
    testlabellist = labellist[i:]
    return traindatalist, trainlabellist,testdatalist,testlabellist


# 6. Tokenisation & Padding --------
def tokenize(data: list, map2num: dict, non_aa_num: int=20) -> list:
    """Tokenize all sequences in a list

    :param data: list of sequences to tokenize
    :type data: list
    :param map2num: ammino acid -> integer token mapping
    :type map2num: dict
    :param non_aa_num: token for non amino acid characters, defaults to 20
    :type non_aa_num: int, optional
    :return: list of tokenized sequences
    :rtype: list
    """
    seq = []
    for count, i in enumerate(data):
        seq.append([map2num.get(j,non_aa_num) for j in list(i)])
    return seq


def truncate_pad(line: list, num_steps: int, padding_token: int) -> list:
    """Truncate or pad a tokenized sequence

    :param line: tokenized sequence
    :type line: list
    :param num_steps: maximum sequence length
    :type num_steps: int
    :param padding_token: token to be used for padding
    :type padding_token: int
    :return: truncated/padded sequence
    :rtype: list
    """
    if len(line) > num_steps:
        return line[:num_steps] # Truncate
    return line + [padding_token] * (num_steps - len(line)) # Pad


def build_seq_array(lines: list, num_steps: int, non_aa_num: int=20) -> torch.tensor:
    """Truncate or pad tokenized sequences and convert to tensor

    :param lines: tokenized sequences
    :type lines: list
    :param num_steps: maximum sequence length
    :type num_steps: int
    :param non_aa_num: token for padding, defaults to 20
    :type non_aa_num: int, optional
    :return: tensor with truncated/padded tokenized sequences
    :rtype: torch.tensor
    """
    return torch.tensor([truncate_pad(l, num_steps, non_aa_num) for l in lines], dtype=torch.long)


# 7. load_array() & load_data()
def load_array(data_arrays: tuple[torch.tensor, torch.tensor], batch_size: int, is_train: bool=True) -> torch.utils.data.DataLoader:
    """Construct a PyTorch data iterator.

    Taken from d2l package"""
    dataset = torch.utils.data.TensorDataset(*data_arrays)
    return torch.utils.data.DataLoader(dataset, batch_size, shuffle=is_train)


def load_data(batch_size: int, num_steps: int, dataset: tuple[list, list]) -> torch.utils.data.DataLoader:
    """Tokenize sequence/label dataset and load into dataloader.

    :param batch_size: size of each batch
    :type batch_size: int
    :param num_steps: maximum sequence length
    :type num_steps: int
    :param dataset: first list contains sequences, second labels
    :type dataset: tuple[list, list]
    :return: torch dataloader which gives a tensor of sequences in a batch and a tensor with their labels
    :rtype: torch.utils.data.DataLoader
    """
    mapaa2num = {aa: i for (i, aa) in enumerate(list("ACDEFGHIKLMNPQRSTVWY"))}
    seq,lab = dataset
    seq = tokenize(seq, mapaa2num)
    seq_array = build_seq_array(seq, num_steps)
    data_arrays = (seq_array, torch.tensor(lab, dtype=torch.long))
    data_iter = load_array(data_arrays, batch_size)
    return data_iter

## Dataset selector





In [4]:
# For the GO dataset
REMOVE = "neg"                   # None | "neg" | "pos"

seq_path  = "expr5Tseq_filtGO_100-1000.lis"
pos_path  = "GO_3A0005739.annotprot"
datalist, labellist = read(seq_path, pos_path)

# Removing half dataset
if REMOVE is not None:
    pos_datalist, neg_datalist = split_labelled(datalist, labellist)
    if REMOVE == "neg":
        neg_datalist = remove_sequences(neg_datalist, 0.5)
    elif REMOVE == "pos":
        pos_datalist = remove_sequences(pos_datalist, 0.5)
    datalist, labellist = fuse_sequence_lists(pos_datalist, neg_datalist)

In [11]:
# For the synthetic dataset
dataset_id = "len100_200_n1000"
REMOVE = None                   # None | "neg" | "pos"

seq_path  = f"{dataset_id}.seq"
pos_path  = f"{dataset_id}.pos"
datalist, labellist = read(seq_path, pos_path)

# Removing half dataset
if REMOVE is not None:
    pos_datalist, neg_datalist = split_labelled(datalist, labellist)
    if REMOVE == "neg":
        neg_datalist = remove_sequences(neg_datalist, 0.8)
    elif REMOVE == "pos":
        pos_datalist = remove_sequences(pos_datalist, 0.8)
    datalist, labellist = fuse_sequence_lists(pos_datalist, neg_datalist)

## Data Loader

In [16]:
batch_size = 10
num_steps = 1000 # max sequence length

traindatalist, trainlabellist, testdatalist, testlabellist = generate_train_test(datalist, labellist, 0.8)
traindataset = [traindatalist, trainlabellist]
testdataset = [testdatalist, testlabellist]

# Set batch_size and num_steps (maximum sequence length)
train_iter = load_data(batch_size, num_steps, traindataset)
test_iter = load_data(batch_size, num_steps, testdataset)

print("batch shape  :", next(iter(train_iter))[0].shape)
print("labels shape :", next(iter(train_iter))[1].shape)

batch shape  : torch.Size([10, 1000])
labels shape : torch.Size([10])


## Training / Evaluation

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
def init_weights(layer):
    if type(layer) == nn.Linear or type(layer) == nn.Conv1d:
        nn.init.xavier_uniform_(layer.weight)

class Trainer:
    def __init__(self, model, loss_fn, optimizer, device='cuda'):
        self.model = model.to(device)
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = device


    # One training epoch -------------------------------------------------------
    def _train_one_epoch(self, train_iter):
        result_loss = 0
        tpos = fpos = tneg = fneg = 0
        self.model.train(True)

        for i, (inputs, labels) in enumerate(train_iter):
            inputs = inputs.to(self.device)
            labels = labels.to(self.device)

            self.optimizer.zero_grad()
            outputs = self.model(inputs)
            loss = self.loss_fn(outputs, labels)
            loss.backward()
            self.optimizer.step()
            result_loss += loss.item()

            # Confusion matrix calculation
            for j, l in enumerate(labels):
                o = outputs[j].tolist().index(max(outputs[j]))
                l = l.item()
                if o == 1 and l == 1:
                    tpos += 1
                elif o == 1 and l == 0:
                    fpos += 1
                elif o == 0 and l == 0:
                    tneg += 1
                elif o == 0 and l == 1:
                    fneg += 1
                else:
                    raise ValueError(f'Output or label is not rounded to zero: {o = } {l = }')

        accuracy = (tpos + tneg) / (tpos + fpos + tneg + fneg)
        precision = tpos / (tpos + fpos) if (tpos + fpos) > 0 else 0
        recall = tpos / (tpos + fneg) if (tpos + fneg) > 0 else 0
        fscore = 2 * tpos / (2*tpos + fpos + fneg) if (2*tpos + fpos + fneg) > 0 else 0

        return accuracy, precision, recall, fscore, result_loss / (i+1)

    # Evaluation epoch ---------------------------------------------------------
    def _test_one_epoch(self, test_iter):
        result_loss = 0
        tpos = fpos = tneg = fneg = 0
        self.model.eval()

        with torch.no_grad():
            for i, (inputs, labels) in enumerate(test_iter, start=1):
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)

                outputs = self.model(inputs)
                loss = self.loss_fn(input=outputs, target=labels)
                result_loss += loss.item()

                # Confusion matrix calculation
                for j, l in enumerate(labels):
                    o = outputs[j].tolist().index(max(outputs[j]))
                    l = l.item()
                    if o == 1 and l == 1:
                        tpos += 1
                    elif o == 1 and l == 0:
                        fpos += 1
                    elif o == 0 and l == 0:
                        tneg += 1
                    elif o == 0 and l == 1:
                        fneg += 1
                    else:
                        raise ValueError(f'Output or label is not rounded to zero: {o = } {l = }')

        accuracy = (tpos + tneg) / (tpos + fpos + tneg + fneg)
        precision = tpos / (tpos + fpos) if (tpos + fpos) > 0 else 0
        recall = tpos / (tpos + fneg) if (tpos + fneg) > 0 else 0
        fscore = 2 * tpos / (2*tpos + fpos + fneg) if (2*tpos + fpos + fneg) > 0 else 0

        return accuracy, precision, recall, fscore, result_loss / (i+1)


    def train(self, epochs, train_iter, test_iter):
        best_acc = 0.0

        print("Current hyper-parameters:", params)
        for epoch in range(1, epochs + 1):
            train_acc, train_prec, train_rec, train_f, train_loss = self._train_one_epoch(train_iter)

            test_acc, test_prec, test_rec, test_f, test_loss = self._test_one_epoch(test_iter)

            print(f"[epoch {epoch:02d}] "
                  f"train-loss={train_loss:.4f} | "
                  f"test-loss={test_loss:.4f} | "
                  f"train-acc={train_acc:.4f} | "
                  f"test-acc={test_acc:.4f} | "
                  f"P={test_prec:.4f} | R={test_rec:.4f} | F1={test_f:.4f}")

            if test_acc >= best_acc:
                best_acc = test_acc

        return best_acc

In [13]:
class BerryCNN1D(nn.Module):
    """1D Convolutional Neural Network for protein function classification"""
    def __init__(self, context_size: int, vocab_size: int = 21,
                 dropout_rate = 0, conv_channels: int = 128,
                 use_bias: bool = False):

        super().__init__()
        assert context_size % 2 == 0, f'Invalid context_size, {context_size} is not an even number'

        self.embedding = nn.Embedding(vocab_size, vocab_size)

        # CNN model for binary classification
        self.conv1 = nn.Sequential(
            # conv block 1
            nn.Conv1d(in_channels=vocab_size, out_channels=conv_channels, kernel_size=3, padding='same', bias=use_bias),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.conv2 = nn.Sequential(
            # conv block 2
            nn.Conv1d(in_channels=conv_channels, out_channels=conv_channels, kernel_size=3, padding='same', bias=use_bias),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1)
        )


        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Sequential(
            # flatten + classification head
            nn.Flatten(1, -1),
            nn.LazyLinear(out_features=2, bias=use_bias)  # binary classification
        )


    def forward(self, x: torch.tensor, targets: torch.tensor = None):
        """Predict protein function class (0 or 1)"""
        x = self.embedding(x).permute(0,2,1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.dropout(x)
        return self.fc(x)


In [14]:
# Selecting the CNN
net = BerryCNN1D(context_size=num_steps, vocab_size=21, conv_channels=128)

## Training the Model

In [15]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    'dropout_rate': [0, 0.3],
    'lr': [0.01, 0.001],
    'momentum': [0, 0.5],
    'conv_channels': [64, 128]
}

grid = list(ParameterGrid(param_grid))

best_acc = 0
best_params = None

for params in grid:
    model = BerryCNN1D(
        context_size=num_steps,
        vocab_size=21,
        dropout_rate=params['dropout_rate'],
        conv_channels=params['conv_channels']
    )
    model.apply(init_weights)

    optimizer = torch.optim.SGD(
        model.parameters(),
        lr=params['lr'],
        momentum=params['momentum']
    )

    loss_fn = nn.CrossEntropyLoss()
    trainer = Trainer(model, loss_fn, optimizer, device)

    acc = trainer.train(epochs=10, train_iter=train_iter, test_iter=test_iter)

    if acc >= best_acc:
        best_acc = []
        best_acc = acc
        best_params = params
        print(f"New best accuracy {best_acc:.4f} with {best_params} \n")

print("Best hyper‑parameters found:", best_params)

Current hyper-parameters: {'conv_channels': 64, 'dropout_rate': 0, 'lr': 0.01, 'momentum': 0}
[epoch 01] train-loss=0.4010 | test-loss=0.4075 | train-acc=0.8660 | test-acc=0.8719 | P=0.0000 | R=0.0000 | F1=0.0000
[epoch 02] train-loss=0.3852 | test-loss=0.3787 | train-acc=0.8694 | test-acc=0.8719 | P=0.0000 | R=0.0000 | F1=0.0000
[epoch 03] train-loss=0.3761 | test-loss=0.3909 | train-acc=0.8694 | test-acc=0.8719 | P=0.0000 | R=0.0000 | F1=0.0000
[epoch 04] train-loss=0.3665 | test-loss=0.3786 | train-acc=0.8694 | test-acc=0.8719 | P=0.0000 | R=0.0000 | F1=0.0000
[epoch 05] train-loss=0.3561 | test-loss=0.3770 | train-acc=0.8694 | test-acc=0.8719 | P=0.0000 | R=0.0000 | F1=0.0000
[epoch 06] train-loss=0.3490 | test-loss=0.3678 | train-acc=0.8697 | test-acc=0.8719 | P=0.0000 | R=0.0000 | F1=0.0000
[epoch 07] train-loss=0.3370 | test-loss=0.4362 | train-acc=0.8704 | test-acc=0.8664 | P=0.4000 | R=0.0860 | F1=0.1416
[epoch 08] train-loss=0.3220 | test-loss=0.4064 | train-acc=0.8728 | test

In [ ]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    'dropout_rate': [0, 0.3],
    'lr': [0.01, 0.001],
    'momentum': [0, 0.5],
    'conv_channels': [64, 128]
}

grid = list(ParameterGrid(param_grid))

best_acc = 0
best_params = None

for params in grid:
    model = BerryCNN1D(
        context_size=num_steps,
        vocab_size=21,
        dropout_rate=params['dropout_rate'],
        conv_channels=params['conv_channels']
    )
    model.apply(init_weights)

    optimizer = torch.optim.SGD(
        model.parameters(),
        lr=params['lr'],
        momentum=params['momentum']
    )

    loss_fn = nn.CrossEntropyLoss()
    trainer = Trainer(model, loss_fn, optimizer, device)

    acc = trainer.train(epochs=10, train_iter=train_iter, test_iter=test_iter)

    if acc >= best_acc:
        best_acc = []
        best_acc = acc
        best_params = params
        print(f"New best accuracy {best_acc:.4f} with {best_params} \n")

print("Best hyper‑parameters found:", best_params)

[epoch 01] train-loss=0.3937 | test-loss=0.4786 | train-acc=0.8722 | test-acc=0.8609 | P=0.0000 | R=0.0000 | F1=0.0000
[epoch 02] train-loss=0.3839 | test-loss=0.3955 | train-acc=0.8722 | test-acc=0.8609 | P=0.0000 | R=0.0000 | F1=0.0000
[epoch 03] train-loss=0.3731 | test-loss=0.3956 | train-acc=0.8722 | test-acc=0.8609 | P=0.0000 | R=0.0000 | F1=0.0000
[epoch 04] train-loss=0.3697 | test-loss=0.4924 | train-acc=0.8722 | test-acc=0.8609 | P=0.0000 | R=0.0000 | F1=0.0000
[epoch 05] train-loss=0.3605 | test-loss=0.3861 | train-acc=0.8722 | test-acc=0.8609 | P=0.0000 | R=0.0000 | F1=0.0000
[epoch 06] train-loss=0.3504 | test-loss=0.3868 | train-acc=0.8725 | test-acc=0.8609 | P=0.0000 | R=0.0000 | F1=0.0000
[epoch 07] train-loss=0.3407 | test-loss=0.4707 | train-acc=0.8728 | test-acc=0.8595 | P=0.4615 | R=0.0594 | F1=0.1053
[epoch 08] train-loss=0.3281 | test-loss=0.4057 | train-acc=0.8725 | test-acc=0.8609 | P=0.0000 | R=0.0000 | F1=0.0000
[epoch 09] train-loss=0.3155 | test-loss=0.4407 

KeyboardInterrupt: 

Verander bovenstaande gegevens in een pandas dataframe



len200_500_n5000nr4 = Best hyper‑parameters found: {'conv_channels': 64, 'dropout_rate': 0, 'lr': 0.01, 'momentum': 0}

len200_500_n5000nr1

Current hyper-parameter: {'conv_channels': 64, 'dropout_rate': 0, 'lr': 0.01, 'momentum': 0}
[epoch 01] train‑loss=0.2816 | test‑loss=0.0206 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 02] train‑loss=0.0108 | test‑loss=0.0062 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.0044 | test‑loss=0.0034 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0027 | test‑loss=0.0023 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0019 | test‑loss=0.0018 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0015 | test‑loss=0.0014 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0012 | test‑loss=0.0012 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0010 | test‑loss=0.0010 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0008 | test‑loss=0.0009 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0007 | test‑loss=0.0008 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 64, 'dropout_rate': 0, 'lr': 0.01, 'momentum': 0}

Current hyper-parameter: {'conv_channels': 64, 'dropout_rate': 0, 'lr': 0.01, 'momentum': 0.5}
[epoch 01] train‑loss=0.6772 | test‑loss=0.1625 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 02] train‑loss=0.0247 | test‑loss=0.0056 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.0040 | test‑loss=0.0027 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0022 | test‑loss=0.0018 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0015 | test‑loss=0.0014 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0011 | test‑loss=0.0012 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0009 | test‑loss=0.0010 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0007 | test‑loss=0.0009 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0006 | test‑loss=0.0008 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0005 | test‑loss=0.0007 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 64, 'dropout_rate': 0, 'lr': 0.01, 'momentum': 0.5}

Current hyper-parameter: {'conv_channels': 64, 'dropout_rate': 0, 'lr': 0.01, 'momentum': 0.9}
[epoch 01] train‑loss=0.0577 | test‑loss=0.0006 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 02] train‑loss=0.0004 | test‑loss=0.0003 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.0002 | test‑loss=0.0002 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0002 | test‑loss=0.0002 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0001 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0001 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0001 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0001 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0001 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0001 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 64, 'dropout_rate': 0, 'lr': 0.01, 'momentum': 0.9}

Current hyper-parameter: {'conv_channels': 64, 'dropout_rate': 0, 'lr': 0.001, 'momentum': 0}
[epoch 01] train‑loss=0.7000 | test‑loss=0.6940 | test‑acc=0.5000 | P=0.5000 | R=0.5000 | F1=0.5000
[epoch 02] train‑loss=0.6864 | test‑loss=0.6803 | test‑acc=0.5610 | P=0.5610 | R=0.5610 | F1=0.5610
[epoch 03] train‑loss=0.6527 | test‑loss=0.6190 | test‑acc=0.7950 | P=0.7950 | R=0.7950 | F1=0.7950
[epoch 04] train‑loss=0.5373 | test‑loss=0.4389 | test‑acc=0.9960 | P=0.9960 | R=0.9960 | F1=0.9960
[epoch 05] train‑loss=0.3297 | test‑loss=0.2355 | test‑acc=0.9980 | P=0.9980 | R=0.9980 | F1=0.9980
[epoch 06] train‑loss=0.1719 | test‑loss=0.1253 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 07] train‑loss=0.0962 | test‑loss=0.0755 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0609 | test‑loss=0.0507 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0425 | test‑loss=0.0371 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0319 | test‑loss=0.0288 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 64, 'dropout_rate': 0, 'lr': 0.001, 'momentum': 0}

Current hyper-parameter: {'conv_channels': 64, 'dropout_rate': 0, 'lr': 0.001, 'momentum': 0.5}
[epoch 01] train‑loss=0.6852 | test‑loss=0.6571 | test‑acc=0.6350 | P=0.6350 | R=0.6350 | F1=0.6350
[epoch 02] train‑loss=0.4732 | test‑loss=0.2545 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.1386 | test‑loss=0.0705 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0485 | test‑loss=0.0323 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0257 | test‑loss=0.0192 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0167 | test‑loss=0.0133 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0121 | test‑loss=0.0100 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0094 | test‑loss=0.0079 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0076 | test‑loss=0.0064 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0063 | test‑loss=0.0054 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 64, 'dropout_rate': 0, 'lr': 0.001, 'momentum': 0.5}

Current hyper-parameter: {'conv_channels': 64, 'dropout_rate': 0, 'lr': 0.001, 'momentum': 0.9}
[epoch 01] train‑loss=0.4575 | test‑loss=0.0568 | test‑acc=0.9980 | P=0.9980 | R=0.9980 | F1=0.9980
[epoch 02] train‑loss=0.0249 | test‑loss=0.0120 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 03] train‑loss=0.0094 | test‑loss=0.0064 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 04] train‑loss=0.0058 | test‑loss=0.0043 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0042 | test‑loss=0.0032 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0031 | test‑loss=0.0026 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0025 | test‑loss=0.0022 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0021 | test‑loss=0.0018 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0018 | test‑loss=0.0015 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0015 | test‑loss=0.0013 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 64, 'dropout_rate': 0, 'lr': 0.001, 'momentum': 0.9}

Current hyper-parameter: {'conv_channels': 64, 'dropout_rate': 0.3, 'lr': 0.01, 'momentum': 0}
[epoch 01] train‑loss=0.4828 | test‑loss=0.0886 | test‑acc=0.9970 | P=0.9970 | R=0.9970 | F1=0.9970
[epoch 02] train‑loss=0.1210 | test‑loss=0.0361 | test‑acc=0.9980 | P=0.9980 | R=0.9980 | F1=0.9980
[epoch 03] train‑loss=0.0842 | test‑loss=0.0165 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0656 | test‑loss=0.0257 | test‑acc=0.9980 | P=0.9980 | R=0.9980 | F1=0.9980
[epoch 05] train‑loss=0.0494 | test‑loss=0.0129 | test‑acc=0.9980 | P=0.9980 | R=0.9980 | F1=0.9980
[epoch 06] train‑loss=0.0432 | test‑loss=0.0124 | test‑acc=0.9980 | P=0.9980 | R=0.9980 | F1=0.9980
[epoch 07] train‑loss=0.0348 | test‑loss=0.0103 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 08] train‑loss=0.0364 | test‑loss=0.0073 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 09] train‑loss=0.0303 | test‑loss=0.0073 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 10] train‑loss=0.0277 | test‑loss=0.0052 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
New best accuracy 1.0000 with {'conv_channels': 64, 'dropout_rate': 0.3, 'lr': 0.01, 'momentum': 0}

Current hyper-parameter: {'conv_channels': 64, 'dropout_rate': 0.3, 'lr': 0.01, 'momentum': 0.5}
[epoch 01] train‑loss=0.3771 | test‑loss=0.0356 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 02] train‑loss=0.0809 | test‑loss=0.0090 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.0433 | test‑loss=0.0102 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 04] train‑loss=0.0257 | test‑loss=0.0057 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0161 | test‑loss=0.0041 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0155 | test‑loss=0.0025 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0104 | test‑loss=0.0019 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0120 | test‑loss=0.0037 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0116 | test‑loss=0.0018 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0066 | test‑loss=0.0023 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 64, 'dropout_rate': 0.3, 'lr': 0.01, 'momentum': 0.5}

Current hyper-parameter: {'conv_channels': 64, 'dropout_rate': 0.3, 'lr': 0.01, 'momentum': 0.9}
[epoch 01] train‑loss=0.1972 | test‑loss=0.0018 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 02] train‑loss=0.0124 | test‑loss=0.0133 | test‑acc=0.9970 | P=0.9970 | R=0.9970 | F1=0.9970
[epoch 03] train‑loss=0.0119 | test‑loss=0.0003 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0123 | test‑loss=0.0013 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0066 | test‑loss=0.0010 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0059 | test‑loss=0.0002 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0035 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0030 | test‑loss=0.0002 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0022 | test‑loss=0.0014 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0030 | test‑loss=0.0003 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 64, 'dropout_rate': 0.3, 'lr': 0.01, 'momentum': 0.9}

Current hyper-parameter: {'conv_channels': 64, 'dropout_rate': 0.3, 'lr': 0.001, 'momentum': 0}
[epoch 01] train‑loss=0.7325 | test‑loss=0.6976 | test‑acc=0.4880 | P=0.4880 | R=0.4880 | F1=0.4880
[epoch 02] train‑loss=0.7319 | test‑loss=0.6953 | test‑acc=0.4900 | P=0.4900 | R=0.4900 | F1=0.4900
[epoch 03] train‑loss=0.7086 | test‑loss=0.6913 | test‑acc=0.5280 | P=0.5280 | R=0.5280 | F1=0.5280
[epoch 04] train‑loss=0.7002 | test‑loss=0.6867 | test‑acc=0.5390 | P=0.5390 | R=0.5390 | F1=0.5390
[epoch 05] train‑loss=0.6927 | test‑loss=0.6684 | test‑acc=0.6040 | P=0.6040 | R=0.6040 | F1=0.6040
[epoch 06] train‑loss=0.6579 | test‑loss=0.6082 | test‑acc=0.8850 | P=0.8850 | R=0.8850 | F1=0.8850
[epoch 07] train‑loss=0.5988 | test‑loss=0.4994 | test‑acc=0.9950 | P=0.9950 | R=0.9950 | F1=0.9950
[epoch 08] train‑loss=0.5019 | test‑loss=0.3739 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 09] train‑loss=0.4247 | test‑loss=0.2741 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.3350 | test‑loss=0.1913 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 64, 'dropout_rate': 0.3, 'lr': 0.001, 'momentum': 0}

Current hyper-parameter: {'conv_channels': 64, 'dropout_rate': 0.3, 'lr': 0.001, 'momentum': 0.5}
[epoch 01] train‑loss=0.7345 | test‑loss=0.6898 | test‑acc=0.5550 | P=0.5550 | R=0.5550 | F1=0.5550
[epoch 02] train‑loss=0.6994 | test‑loss=0.6511 | test‑acc=0.7460 | P=0.7460 | R=0.7460 | F1=0.7460
[epoch 03] train‑loss=0.5778 | test‑loss=0.3980 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.3361 | test‑loss=0.1642 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.1984 | test‑loss=0.0937 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.1398 | test‑loss=0.0594 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.1103 | test‑loss=0.0387 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0840 | test‑loss=0.0340 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0726 | test‑loss=0.0237 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0676 | test‑loss=0.0212 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 64, 'dropout_rate': 0.3, 'lr': 0.001, 'momentum': 0.5}

Current hyper-parameter: {'conv_channels': 64, 'dropout_rate': 0.3, 'lr': 0.001, 'momentum': 0.9}
[epoch 01] train‑loss=0.5959 | test‑loss=0.1251 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 02] train‑loss=0.1392 | test‑loss=0.0384 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.0771 | test‑loss=0.0157 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0641 | test‑loss=0.0146 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0485 | test‑loss=0.0105 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0340 | test‑loss=0.0073 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0311 | test‑loss=0.0056 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0277 | test‑loss=0.0069 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0273 | test‑loss=0.0056 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0219 | test‑loss=0.0082 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 64, 'dropout_rate': 0.3, 'lr': 0.001, 'momentum': 0.9}

Current hyper-parameter: {'conv_channels': 128, 'dropout_rate': 0, 'lr': 0.01, 'momentum': 0}
[epoch 01] train‑loss=0.2720 | test‑loss=0.0191 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 02] train‑loss=0.0099 | test‑loss=0.0059 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.0041 | test‑loss=0.0033 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0026 | test‑loss=0.0022 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0018 | test‑loss=0.0017 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0014 | test‑loss=0.0014 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0011 | test‑loss=0.0011 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0010 | test‑loss=0.0010 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0008 | test‑loss=0.0008 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0007 | test‑loss=0.0007 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 128, 'dropout_rate': 0, 'lr': 0.01, 'momentum': 0}

Current hyper-parameter: {'conv_channels': 128, 'dropout_rate': 0, 'lr': 0.01, 'momentum': 0.5}
[epoch 01] train‑loss=0.1823 | test‑loss=0.0073 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 02] train‑loss=0.0042 | test‑loss=0.0029 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.0020 | test‑loss=0.0018 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0013 | test‑loss=0.0014 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0009 | test‑loss=0.0011 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0007 | test‑loss=0.0010 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0006 | test‑loss=0.0009 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0005 | test‑loss=0.0008 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0004 | test‑loss=0.0007 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0004 | test‑loss=0.0006 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 128, 'dropout_rate': 0, 'lr': 0.01, 'momentum': 0.5}

Current hyper-parameter: {'conv_channels': 128, 'dropout_rate': 0, 'lr': 0.01, 'momentum': 0.9}
[epoch 01] train‑loss=0.0777 | test‑loss=0.0004 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 02] train‑loss=0.0003 | test‑loss=0.0002 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.0002 | test‑loss=0.0002 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0001 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0001 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0001 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0001 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0001 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0001 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0000 | test‑loss=0.0000 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 128, 'dropout_rate': 0, 'lr': 0.01, 'momentum': 0.9}

Current hyper-parameter: {'conv_channels': 128, 'dropout_rate': 0, 'lr': 0.001, 'momentum': 0}
[epoch 01] train‑loss=0.6962 | test‑loss=0.6926 | test‑acc=0.5190 | P=0.5190 | R=0.5190 | F1=0.5190
[epoch 02] train‑loss=0.6822 | test‑loss=0.6718 | test‑acc=0.5710 | P=0.5710 | R=0.5710 | F1=0.5710
[epoch 03] train‑loss=0.6298 | test‑loss=0.5712 | test‑acc=0.9840 | P=0.9840 | R=0.9840 | F1=0.9840
[epoch 04] train‑loss=0.4671 | test‑loss=0.3544 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.2563 | test‑loss=0.1787 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.1314 | test‑loss=0.0979 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0762 | test‑loss=0.0607 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0501 | test‑loss=0.0421 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0360 | test‑loss=0.0314 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0276 | test‑loss=0.0247 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 128, 'dropout_rate': 0, 'lr': 0.001, 'momentum': 0}

Current hyper-parameter: {'conv_channels': 128, 'dropout_rate': 0, 'lr': 0.001, 'momentum': 0.5}
[epoch 01] train‑loss=0.5885 | test‑loss=0.3798 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 02] train‑loss=0.1963 | test‑loss=0.0867 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.0562 | test‑loss=0.0362 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0278 | test‑loss=0.0206 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0176 | test‑loss=0.0140 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0127 | test‑loss=0.0104 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0098 | test‑loss=0.0082 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0079 | test‑loss=0.0068 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0066 | test‑loss=0.0057 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0056 | test‑loss=0.0049 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 128, 'dropout_rate': 0, 'lr': 0.001, 'momentum': 0.5}

Current hyper-parameter: {'conv_channels': 128, 'dropout_rate': 0, 'lr': 0.001, 'momentum': 0.9}
[epoch 01] train‑loss=0.3794 | test‑loss=0.0236 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 02] train‑loss=0.0112 | test‑loss=0.0058 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.0043 | test‑loss=0.0031 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0026 | test‑loss=0.0021 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0018 | test‑loss=0.0015 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0014 | test‑loss=0.0012 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0011 | test‑loss=0.0010 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0009 | test‑loss=0.0008 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0008 | test‑loss=0.0007 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0007 | test‑loss=0.0006 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 128, 'dropout_rate': 0, 'lr': 0.001, 'momentum': 0.9}

Current hyper-parameter: {'conv_channels': 128, 'dropout_rate': 0.3, 'lr': 0.01, 'momentum': 0}
[epoch 01] train‑loss=0.3985 | test‑loss=0.0512 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 02] train‑loss=0.0630 | test‑loss=0.0151 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 03] train‑loss=0.0286 | test‑loss=0.0088 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0243 | test‑loss=0.0079 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0147 | test‑loss=0.0068 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 06] train‑loss=0.0142 | test‑loss=0.0037 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0091 | test‑loss=0.0028 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0095 | test‑loss=0.0046 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0074 | test‑loss=0.0035 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0057 | test‑loss=0.0019 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 128, 'dropout_rate': 0.3, 'lr': 0.01, 'momentum': 0}

Current hyper-parameter: {'conv_channels': 128, 'dropout_rate': 0.3, 'lr': 0.01, 'momentum': 0.5}
[epoch 01] train‑loss=0.4277 | test‑loss=0.0294 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 02] train‑loss=0.0735 | test‑loss=0.0119 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.0468 | test‑loss=0.0097 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0389 | test‑loss=0.0136 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 05] train‑loss=0.0256 | test‑loss=0.0033 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0292 | test‑loss=0.0068 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 07] train‑loss=0.0252 | test‑loss=0.0052 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 08] train‑loss=0.0151 | test‑loss=0.0030 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0200 | test‑loss=0.0114 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 10] train‑loss=0.0120 | test‑loss=0.0042 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
New best accuracy 1.0000 with {'conv_channels': 128, 'dropout_rate': 0.3, 'lr': 0.01, 'momentum': 0.5}

Current hyper-parameter: {'conv_channels': 128, 'dropout_rate': 0.3, 'lr': 0.01, 'momentum': 0.9}
[epoch 01] train‑loss=0.1308 | test‑loss=0.0009 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 02] train‑loss=0.0027 | test‑loss=0.0009 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.0031 | test‑loss=0.0004 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0029 | test‑loss=0.0007 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0015 | test‑loss=0.0009 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0021 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0018 | test‑loss=0.0004 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0003 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0003 | test‑loss=0.0003 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0002 | test‑loss=0.0001 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 128, 'dropout_rate': 0.3, 'lr': 0.01, 'momentum': 0.9}

Current hyper-parameter: {'conv_channels': 128, 'dropout_rate': 0.3, 'lr': 0.001, 'momentum': 0}
[epoch 01] train‑loss=0.7231 | test‑loss=0.6839 | test‑acc=0.5180 | P=0.5180 | R=0.5180 | F1=0.5180
[epoch 02] train‑loss=0.6820 | test‑loss=0.6185 | test‑acc=0.8370 | P=0.8370 | R=0.8370 | F1=0.8370
[epoch 03] train‑loss=0.5689 | test‑loss=0.4380 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.3908 | test‑loss=0.2455 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.2549 | test‑loss=0.1381 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.1779 | test‑loss=0.0833 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.1265 | test‑loss=0.0555 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0996 | test‑loss=0.0410 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0812 | test‑loss=0.0308 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0678 | test‑loss=0.0267 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 128, 'dropout_rate': 0.3, 'lr': 0.001, 'momentum': 0}

Current hyper-parameter: {'conv_channels': 128, 'dropout_rate': 0.3, 'lr': 0.001, 'momentum': 0.5}
[epoch 01] train‑loss=0.7298 | test‑loss=0.6833 | test‑acc=0.6080 | P=0.6080 | R=0.6080 | F1=0.6080
[epoch 02] train‑loss=0.6891 | test‑loss=0.6188 | test‑acc=0.8530 | P=0.8530 | R=0.8530 | F1=0.8530
[epoch 03] train‑loss=0.5337 | test‑loss=0.3247 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.2705 | test‑loss=0.1130 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.1295 | test‑loss=0.0547 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0829 | test‑loss=0.0312 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 07] train‑loss=0.0566 | test‑loss=0.0212 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0472 | test‑loss=0.0175 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0367 | test‑loss=0.0158 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0360 | test‑loss=0.0138 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 128, 'dropout_rate': 0.3, 'lr': 0.001, 'momentum': 0.5}

Current hyper-parameter: {'conv_channels': 128, 'dropout_rate': 0.3, 'lr': 0.001, 'momentum': 0.9}
[epoch 01] train‑loss=0.5881 | test‑loss=0.1344 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 02] train‑loss=0.1153 | test‑loss=0.0155 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 03] train‑loss=0.0448 | test‑loss=0.0081 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 04] train‑loss=0.0251 | test‑loss=0.0087 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 05] train‑loss=0.0208 | test‑loss=0.0072 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 06] train‑loss=0.0184 | test‑loss=0.0087 | test‑acc=0.9990 | P=0.9990 | R=0.9990 | F1=0.9990
[epoch 07] train‑loss=0.0109 | test‑loss=0.0031 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 08] train‑loss=0.0088 | test‑loss=0.0026 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 09] train‑loss=0.0119 | test‑loss=0.0020 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
[epoch 10] train‑loss=0.0066 | test‑loss=0.0017 | test‑acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000
New best accuracy 1.0000 with {'conv_channels': 128, 'dropout_rate': 0.3, 'lr': 0.001, 'momentum': 0.9}

Best hyper‑parameters found: {'conv_channels': 128, 'dropout_rate': 0.3, 'lr': 0.001, 'momentum': 0.9}

{'conv_channels': 128, 'dropout_rate': 0.3, 'lr': 0.001, 'momentum': 0.9}
